# Practical 04 - Bug2

In [ ]:
from Practical04_Support.Obstacle import *
from Practical04_Support.path_animation import *

import meshcat.geometry as g
import meshcat.transformations as tf

from ece4078.Utility import StartMeshcat
from shapely import geometry

In [ ]:
vis = StartMeshcat()


## Helper functions
These are the helper functions used by Bug2 when looking for a path between the start and goal locations:

- ``is_about_to_hit_obstacle(.)``, this method searches for the obstacle that is the closest to the robot and verifies whether the robot's touch sensor has been activated. This method depends on ``find_closest_obstacle(.)``.

- ``go_back_to_goal(.)``, this method determines whether the robot should stop circumnavigating the obstacle and move towards the goal instead.

- ``has_reached_goal(.)``, this method tells us whether the robot has reached the goal

- ``move_towards_goal(.)``, this method moves the robot along the line that connects the start and goal positions

In [ ]:
# Helper functions
def find_closest_obstacle(position, obstacle_list, ccw):
    results = [obs.compute_distance_point_to_polygon(position, ccw) for obs in obstacle_list]
    closest_obs = np.argmin([v[0] for v in results])
    return obstacles[closest_obs], results[closest_obs]

def has_reached_goal(current_pos, goal, step_size, tolerance):
    if compute_distance_between_points(current_pos, goal) > step_size + tolerance:
        return False
    return True

def move_towards_goal(current_pos, goal_line, goal_pos, initial_robot_pos, step_size):
    direction_to_goal = get_direction_from_line(goal_line)
    start_goal = np.array(goal_pos) - np.array(initial_robot_pos)
    start_current = np.array(current_pos) - np.array(initial_robot_pos)
    dir = 1 if np.linalg.norm(start_goal) > (np.dot(start_current, start_goal) / np.linalg.norm(start_goal)) else -1
    next_position = current_pos + step_size * direction_to_goal * dir
    return next_position


def is_about_to_hit_obstacle(next_pos, obstacle_list, step_size, ccw):
    obs, (_, _) = find_closest_obstacle(next_pos, obstacle_list, ccw)
    point = geometry.Point(next_pos)
    polygon = geometry.Polygon(obs.vertices)
    if point.within(polygon) or point.touches(polygon):
        return True
    else:
        return False

def go_back_to_goal(next_pos, goal_pos, initial_robot_pos, start_to_goal_line, distance_to_hit_point, step_size, segment_not_line = False):
    a, b, c = start_to_goal_line
    dist_to_robot_goal_line = np.abs(a*next_pos[0] + b*next_pos[1] - c)/math.sqrt(a*a + b*b)
    new_dist_to_goal = compute_distance_between_points(next_pos, goal_pos)
    stop_following_obstacle = (dist_to_robot_goal_line <= step_size) and (new_dist_to_goal < distance_to_hit_point)

    # This is to change if you consider goal line a line or a segment
    # The behaviour will be different depending on the scenario
    if segment_not_line:
        start_goal = np.array(goal_pos) - np.array(initial_robot_pos)
        start_current = np.array(next_pos) - np.array(initial_robot_pos)
        within_segment = np.linalg.norm(start_goal) > (np.dot(start_current, start_goal) / np.linalg.norm(start_goal))
        stop_following_obstacle = stop_following_obstacle and within_segment
        
    return stop_following_obstacle

## Main Algorithm

Below we show the implementation of the Bug2 Algorithm. This implementation is based on the pseudocode presented in the lecture

![title](https://i.postimg.cc/zXq09b5t/bug2.png)

In [ ]:
def bug2_algorithm(goal_pos,initial_robot_pos,robot_step_size,obstacles, ccw, tolerance = 0.01):
    # Start algorithm
    robot_path = [initial_robot_pos]
    current_robot_pos = initial_robot_pos
    robot_to_goal_line = compute_line_through_points(current_robot_pos, goal_pos)

    # While goal not reached
    while not has_reached_goal(current_robot_pos, goal_pos, robot_step_size, tolerance):

        # Move towards goal
        next_robot_pos = move_towards_goal(current_robot_pos, robot_to_goal_line, goal_pos, initial_robot_pos, robot_step_size)
        if is_about_to_hit_obstacle(next_robot_pos, obstacles, robot_step_size, ccw):

            # Go back to current position
            next_robot_pos = np.copy(current_robot_pos)
            # Compute distance from hit point to goal
            hit_dist_to_goal = compute_distance_between_points(current_robot_pos, goal_pos)
            while True:
                # Find nearest_obstacle and start navigating around 
                closest_obs, (closest_obs_distance, obst_segment) = find_closest_obstacle(next_robot_pos,
                                                                                        obstacles, ccw)
                # Get direction along obstacle
                direction_around_obstacle = closest_obs.compute_tangent_vector_to_polygon(next_robot_pos, 
                                                                                        obst_segment)
                # Move along obstacle
                next_robot_pos = next_robot_pos + robot_step_size * direction_around_obstacle
                    
                robot_path.append(next_robot_pos)

                # Stop getting around obstacle when:
                # * We are back to the original line
                # * We are closer to the goal than when we started getting around obstacle.
                # * Segment_not_line=True meaning the robot will only stop getting around the obstacle
                #   when it hits the segment again, not the extended segment (a line).
                stop_getting_around = go_back_to_goal(next_robot_pos, goal_pos, initial_robot_pos, robot_to_goal_line,
                                                        hit_dist_to_goal, robot_step_size, segment_not_line=False)

                if stop_getting_around:
                    break

        # Update current state and add to path
        current_robot_pos = next_robot_pos
        robot_path.append(current_robot_pos)
        
    path = np.array(robot_path)
    return path

## Bug2 in Execution

Let's see the Bug2 algorithm in execution. We consider three cases:

- Two obstacles
- Two obstacles with reversed start and goal locations (manually switch the definition of the start and goal locations)
- A continuos obstacle (see picture in the FLUX question)

**Interaction**:
- Swap start and goal location as well as the obstacles. Re-run the previous cell and observe the paths generated for both cases. Is the robot moving on left or the right?

In [ ]:
#Set parameters

# Order A
goal_pos = [20.0, 10.0]
initial_robot_pos = [25.0, 40.0]

# Order B
# Uncomment the next two lines to see the bug algorithm in reverse order
# goal_pos = [25.0, 40.0]
# initial_robot_pos = [20.0, 10.0]

# Robot Step Size and turning behaviour (counter clockwise (ccw) or clockwise)
robot_step_size = 0.5
ccw = True

# Obstacle configuration
obstacle1 = Rectangle(origin=np.array([10, 20]), width=30, height=5)
obstacle2 = Polygon(vertices= np.array([[20, 35], [35, 35], [35, 50], [40, 50],
                                       [40, 30], [20, 30]]))
obstacles = [obstacle1, obstacle2]


# ---------------------------------- Parameters associated to 2nd FLUX Question ----------------------
# initial_robot_pos = [25.0, 5.0]
# goal_pos = [25.0, 40.0]  
# robot_step_size = 0.3

# ccw = False

# # Create obstacles
# obstacle3 = Polygon(vertices= np.array([[10, 20], [35, 20], [35, 50], [15, 50],
#                                        [15, 35], [28, 35], [28, 30], [10, 30],
#                                        [10, 55], [40, 55], [40, 15], [10, 15]]))
# obstacles = [obstacle3]
# ----------------------------------------------------------------------------------------------------

# The code below finds the path using bug2 algorithm and animate it
path = bug2_algorithm(goal_pos,initial_robot_pos,robot_step_size,obstacles, ccw, tolerance = 0.02)
steps = len(path)

In [ ]:
thickness = 0.5
robot_size = 0.5
vis.delete()
scale = 60
vis.Set2DView(scale, center = [lim for lim in [-30, 60, 60, 0]])
print("animating ...")
animate_path_bug(vis,initial_robot_pos,goal_pos,path,obstacles, robot_size, thickness)

import ipywidgets as widgets
msg = widgets.HTML(f"<big>Total of steps: {steps}</big>")
display(widgets.VBox([msg, vis.show_inline(height = 500)]))